In [30]:
import sys
import pandas
import numpy as np
import matplotlib.pyplot as plt
import chess
import chess.svg
from string import Template
import re

attacker = "black"

if attacker == "white":
    cluster_representatives = pandas.read_csv("processed_data/white_cluster_representatives_with_analysis.tsv", sep="\t")
    filtered_data = pandas.read_csv("processed_data/white_filtered_data_san_fen_cluster.tsv", sep="\t")
    img_directory = "../docs/assets/white_svg_boards"
    board_orientation = chess.WHITE
    webpage_content_filename = "processed_data/white_webpage_content.txt"
if attacker == "black":
    cluster_representatives = pandas.read_csv("processed_data/black_cluster_representatives_with_analysis.tsv", sep="\t")
    filtered_data = pandas.read_csv("processed_data/black_filtered_data_san_fen_cluster.tsv", sep="\t")
    board_orientation = chess.BLACK
    img_directory = "../docs/assets/black_svg_boards"
    webpage_content_filename = "processed_data/black_webpage_content.txt"

In [6]:
# Render chess board images from the first element of each cluster

board_colors = chess.svg.DEFAULT_COLORS.copy()
board_colors["square light"] = "#f0d9b5"
board_colors["square dark"] = "#b58863"
board_colors["square light lastmove"] = "#cdd26a"
board_colors["square dark lastmove"] = "#aaa23b"

for i in range(cluster_representatives.shape[0]):
    board = chess.Board()
    moves = cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("UCI_moves")].split(",")
    error_occurred = False
    last_move = ""
    for move in moves:
        board.push_uci(move)
        last_move = move
    boardsvg = chess.svg.board(board, orientation=board_orientation, lastmove=chess.Move.from_uci(last_move), colors=board_colors)
    outputfile = open(img_directory + "/board_" + str(cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("cluster_idx")]) + ".svg", "w")
    outputfile.write(boardsvg)
    outputfile.close()

In [31]:
# Render horizontal barplots of win proportions from the first element of each cluster

for i in range(cluster_representatives.shape[0]):
    num_games = cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("white_wins")] + cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("draws")] + cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("black_wins")]
    current_row = cluster_representatives.iloc[[i]]
    current_row = current_row[["white_win_prob", "draw_prob", "black_win_prob"]]
    current_row = current_row * 100

    current_row.plot(
        kind = 'barh',
        stacked = True,
        legend=None,
        xlim=(0,100),
        color = ["white", "gray", "black"],
        edgecolor="black",
        figsize=(5, 0.75))

    plt.axis("off")

    cs = current_row.cumsum(1)
    current_row = current_row.values.flatten()
    cs = cs.values.flatten()
    plt.text(50, 0.5, "Win Proportions, Lichess Games", va="center", ha="center", fontsize=12)
    plt.text(50, -0.5, "Number of games: " + str(num_games), va="center", ha="center")
    for j in range(3):
        text_color = "black"
        if j == 2:
            text_color = "white"
        if current_row[j] > 15:
            plt.text((cs - current_row / 2)[j], 0, str(np.round(current_row[j], 1)) + "%", va="center", ha="center", color = text_color)
    plt.savefig(img_directory + "/barplot_" + str(cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("cluster_idx")]) + ".svg", format="svg", bbox_inches="tight", transparent=True)
    plt.close()

In [2]:
# Get webpage content and print to file webpage_content_filename
# Create webpage content output file, overwriting if exists
open(webpage_content_filename, 'w').close()

template = Template("""#### Trap #${cluster_idx}, ${opening}

${san}
<br>
<a target="_blank" rel="noopener noreferrer" href="${analysis_url}">
<img src = "{{site.baseurl}}/assets/${attacker}_svg_boards/board_${cluster_idx}.svg" alt="Chess Board for Trap #${cluster_idx}" width="350"/>
</a>
<br>
<img src = "{{site.baseurl}}/assets/${attacker}_svg_boards/barplot_${cluster_idx}.svg" alt="Win-Loss plot for Trap #${cluster_idx}" width="350"/>
<br>
Probability of opponent playing into this position (excluding first move): ${prob_trimmed}%
<br>
Probability of opponent playing into this position (including first move): ${prob}%
<br>
Stockfish evaluation: ${stockfish_eval}
<br>
Analysis board: <a target="_blank" rel="noopener noreferrer" href="${analysis_url}">${analysis_url}</a>


""")

for i in range(100):
    cluster_idx = cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("cluster_idx")]
    opening = cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("Opening")]
    opening = re.sub("[A-Z][0-9][0-9]: ", "", opening)
    san = cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("san")]
    san = re.sub("1\\.", "1\\.", san)
    prob_trimmed = cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("prob_trimmed")]
    prob_trimmed = str(round(prob_trimmed * 100, 1))
    prob = cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("prob")]
    prob = str(round(prob * 100, 1))
    stockfish_eval = cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("stockfish_eval")]
    # Will have to handle the situation if this is a checkmate string rather than an integer
    stockfish_eval_integer = int(stockfish_eval)
    stockfish_eval = str(round(stockfish_eval_integer / 100, 1))
    if stockfish_eval_integer > 0:
        stockfish_eval = "+" + stockfish_eval
    if stockfish_eval_integer < -150:
        stockfish_eval = stockfish_eval + ". Better for Black with perfect play"
    elif stockfish_eval_integer < -50:
        stockfish_eval = stockfish_eval + ". Slightly better for Black with perfect play"
    elif stockfish_eval_integer < 50:
        stockfish_eval = stockfish_eval + ". Roughly even"
    elif stockfish_eval_integer < 150:
        stockfish_eval = stockfish_eval + ". Slightly better for White with perfect play"
    else:
        stockfish_eval = stockfish_eval + ". Better for White with perfect play"
    
    analysis_url = cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("analysis_url")]
    move_index = cluster_representatives.iloc[i, cluster_representatives.columns.get_loc("move_index")]
    analysis_url = analysis_url + "/" + attacker + "#" + str(move_index)
    
    vals = {"cluster_idx": cluster_idx, "opening": opening, "san": san, "prob_trimmed": prob_trimmed, "prob": prob, "stockfish_eval": stockfish_eval, "analysis_url": analysis_url, "attacker": attacker}
    new_content = template.safe_substitute(vals)
    with open(webpage_content_filename, "a") as myfile:
	    myfile.write(new_content + "\n")